In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam

data = pd.read_excel('powerball.xlsx')
next_date = input("Enter the date for the next prediction (MM/DD/YYYY): ")

sequence_length = 7

data_copy = data.copy()
data.drop(['Date'], axis=1, inplace=True)

feature_count = data.shape[1]

training_data = data.copy()

training_rows = training_data.values.shape[0]
training_samples = np.empty([training_rows - sequence_length, sequence_length, feature_count], dtype=float)
training_labels = np.empty([training_rows - sequence_length, feature_count], dtype=float)
for i in range(0, training_rows - sequence_length):
    training_samples[i] = training_data.iloc[i: i + sequence_length, 0: feature_count]
    training_labels[i] = training_data.iloc[i + sequence_length: i + sequence_length + 1, 0: feature_count]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(training_data.values)
scaled_training_samples = pd.DataFrame(data=scaled_data, index=training_data.index)

x_train = np.empty([training_rows - sequence_length, sequence_length, feature_count], dtype=float)
y_train = np.empty([training_rows - sequence_length, feature_count], dtype=float)

for i in range(0, training_rows - sequence_length):
    x_train[i] = scaled_training_samples.iloc[i: i + sequence_length, 0: feature_count]
    y_train[i] = scaled_training_samples.iloc[i + sequence_length: i + sequence_length + 1, 0: feature_count]

rnn_model = Sequential()
rnn_model.add(Bidirectional(LSTM(240, input_shape=(sequence_length, feature_count), return_sequences=True)))
rnn_model.add(Dropout(0.2))
rnn_model.add(Bidirectional(LSTM(240, input_shape=(sequence_length, feature_count), return_sequences=True)))
rnn_model.add(Dropout(0.2))
rnn_model.add(Bidirectional(LSTM(240, input_shape=(sequence_length, feature_count), return_sequences=True)))
rnn_model.add(Bidirectional(LSTM(240, input_shape=(sequence_length, feature_count), return_sequences=False)))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(70))
rnn_model.add(Dense(feature_count))

rnn_model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])

rnn_model.fit(x=x_train, y=y_train, batch_size=100, epochs=1200, verbose=2)

print('-' * 40)
print('Prediction vs. Ground Truth without rounding up or down')
for i in range(1, 10):
    test_data = data_copy.copy()
    test_data = test_data.tail((sequence_length + 10 - i))
    test_data = test_data.head((sequence_length + 1))
    test_date = data_copy.iloc[(test_data.tail().index[-1])]['Date']
    test_input = test_data.head((sequence_length))
    test_input.drop(['Date'], axis=1, inplace=True)
    test_input = np.array(test_input)
    x_test = scaler.transform(test_input)
    y_test_pred = rnn_model.predict(np.array([x_test]))
    test_data.drop(['Date'], axis=1, inplace=True)
    y_test_true = test_data.tail(1)
    print('Drawing Date:', test_date)
    print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0])
    print('Ground Truth:\t', np.array(y_test_true)[0])
    print('-' * 40)

print('-' * 40)
print('Prediction vs. Ground Truth with rounding up')
for i in range(1, 10):
    test_data = data_copy.copy()
    test_data = test_data.tail((sequence_length + 10 - i))
    test_data = test_data.head((sequence_length + 1))
    test_date = data_copy.iloc[(test_data.tail().index[-1])]['Date']
    test_input = test_data.head((sequence_length))
    test_input.drop(['Date'], axis=1, inplace=True)
    test_input = np.array(test_input)
    x_test = scaler.transform(test_input)
    y_test_pred = rnn_model.predict(np.array([x_test]))
    test_data.drop(['Date'], axis=1, inplace=True)
    y_test_true = test_data.tail(1)
    print('Drawing Date:', test_date)
    print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0] + 1)
    print('Ground Truth:\t', np.array(y_test_true)[0])
    print('-' * 40)



print('-' * 40)
print('Predict the Future Drawing')
future_data = data.copy()
future_data = future_data.tail((sequence_length))
future_data = np.array(future_data)
x_future = scaler.transform(future_data)
y_future_pred = rnn_model.predict(np.array([x_future]))
print('Drawing Date:', next_date)
print('Prediction without rounding up or down:\t', scaler.inverse_transform(y_future_pred).astype(int)[0])
print('Prediction with rounding up           :\t', scaler.inverse_transform(y_future_pred).astype(int)[0] + 1)
print('Prediction with rounding down         :\t', scaler.inverse_transform(y_future_pred).astype(int)[0] - 1)
print('-' * 40)

Epoch 1/1200
15/15 - 28s - loss: 0.9957 - accuracy: 0.2276 - 28s/epoch - 2s/step
Epoch 2/1200
15/15 - 8s - loss: 0.9844 - accuracy: 0.2534 - 8s/epoch - 508ms/step
Epoch 3/1200
15/15 - 9s - loss: 0.9815 - accuracy: 0.2541 - 9s/epoch - 614ms/step
Epoch 4/1200
15/15 - 9s - loss: 0.9805 - accuracy: 0.2432 - 9s/epoch - 572ms/step
Epoch 5/1200
15/15 - 8s - loss: 0.9795 - accuracy: 0.2493 - 8s/epoch - 530ms/step
Epoch 6/1200
15/15 - 9s - loss: 0.9786 - accuracy: 0.2439 - 9s/epoch - 590ms/step
Epoch 7/1200
15/15 - 9s - loss: 0.9784 - accuracy: 0.2561 - 9s/epoch - 571ms/step
Epoch 8/1200
15/15 - 8s - loss: 0.9770 - accuracy: 0.2493 - 8s/epoch - 551ms/step
Epoch 9/1200
15/15 - 9s - loss: 0.9770 - accuracy: 0.2507 - 9s/epoch - 595ms/step
Epoch 10/1200
15/15 - 8s - loss: 0.9771 - accuracy: 0.2459 - 8s/epoch - 561ms/step
Epoch 11/1200
15/15 - 8s - loss: 0.9762 - accuracy: 0.2520 - 8s/epoch - 562ms/step
Epoch 12/1200
15/15 - 9s - loss: 0.9763 - accuracy: 0.2595 - 9s/epoch - 592ms/step
Epoch 13/1200


<ipython-input-2-3b361717a7f4>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)


1/1 [==============================] - 3s 3s/step
Drawing Date: 2023-10-05 00:00:00
Prediction:	 [23 54 69 20 32  2]
Ground Truth:	 [24 55 69 21 33  3]
----------------------------------------
1/1 [==============================] - 0s 35ms/step
Drawing Date: 5/13/2023
Prediction:	 [ 2 46 20 22 15 11]
Ground Truth:	 [ 3 46 20 23 15 11]
----------------------------------------
1/1 [==============================] - 0s 34ms/step
Drawing Date: 5/15/2023
Prediction:	 [58  0 26 55 27 25]
Ground Truth:	 [58  1 26 55 28 25]
----------------------------------------


<ipython-input-2-3b361717a7f4>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)
<ipython-input-2-3b361717a7f4>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)
<ipython-input-2-3b361717a7f4>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)


1/1 [==============================] - 0s 33ms/step
Drawing Date: 5/17/2023
Prediction:	 [50 17 36 45 34 14]
Ground Truth:	 [51 18 37 45 34 14]
----------------------------------------
1/1 [==============================] - 0s 31ms/step
Drawing Date: 5/20/2023
Prediction:	 [23 63 32 38 17 23]
Ground Truth:	 [23 63 32 38 17 23]
----------------------------------------
1/1 [==============================] - 0s 31ms/step
Drawing Date: 5/22/2023
Prediction:	 [37 51 48  9 67 24]
Ground Truth:	 [38 52 48  9 68 25]
----------------------------------------


...

1/1 [==============================] - 0s 34ms/step
Drawing Date: 5/17/2023
Prediction:	 [51 18 37 46 35 15]
Ground Truth:	 [51 18 37 45 34 14]
----------------------------------------
1/1 [==============================] - 0s 31ms/step
Drawing Date: 5/20/2023
Prediction:	 [24 64 33 39 18 24]
Ground Truth:	 [23 63 32 38 17 23]
----------------------------------------
1/1 [==============================] - 0s 30ms/step
Drawing Date: 5/22/2023
Prediction:	 [38 52 49 10 68 25]
Ground Truth:	 [38 52 48  9 68 25]
----------------------------------------


<ipython-input-2-3b361717a7f4>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)
<ipython-input-2-3b361717a7f4>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)
<ipython-input-2-3b361717a7f4>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)


1/1 [==============================] - 0s 32ms/step
Drawing Date: 5/24/2023
Prediction:	 [58 45 22 50 12 27]
Ground Truth:	 [58 44 21 50 12 26]
----------------------------------------
1/1 [==============================] - 0s 34ms/step
Drawing Date: 5/27/2023
Prediction:	 [48 40 58 36 25  4]
Ground Truth:	 [48 39 56 38 24  4]
----------------------------------------
1/1 [==============================] - 0s 32ms/step
Drawing Date: 5/29/2023
Prediction:	 [64 35 22 63 33 25]
Ground Truth:	 [64 35 21 62 33 24]
----------------------------------------
----------------------------------------
Predict the Future Drawing


<ipython-input-2-3b361717a7f4>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)
<ipython-input-2-3b361717a7f4>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_input.drop(['Date'], axis=1, inplace=True)


1/1 [==============================] - 0s 33ms/step
Drawing Date: 06/07/2023
Prediction without rounding up or down:	 [38 43 44 24 58  9]
Prediction with rounding up           :	 [39 44 45 25 59 10]
Prediction with rounding down         :	 [37 42 43 23 57  8]
----------------------------------------
